In [1]:
from analysis.eu_gdp_vs_ntl import *
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
gdp = load_prep()

In [3]:
gdp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19 entries, 2000 to 2018
Columns: 2026 entries, EU27_2020 to TRC34
dtypes: float64(2026)
memory usage: 300.9+ KB


In [4]:
gdp

GEO/TIME,EU27_2020,EU28,BE,BE1,BE10,BE100,BE2,BE21,BE211,BE212,...,TRC12,TRC13,TRC2,TRC21,TRC22,TRC3,TRC31,TRC32,TRC33,TRC34
2000,7869519.31,9667201.31,256376.4,48685.94,48685.94,48685.94,146316.66,48308.76,29659.56,7523.93,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001,8241989.57,10074332.57,264334.9,50781.94,50781.94,50781.94,150557.27,48932.02,30094.79,7751.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002,8539137.11,10432496.11,273255.9,52715.02,52715.02,52715.02,155835.49,51211.39,31205.33,8342.27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003,8767872.85,10585057.85,281200.2,53585.68,53585.68,53585.68,160912.03,52538.38,31852.14,8740.92,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004,9167951.15,11113149.15,296819.7,56382.87,56382.87,56382.87,170128.85,56056.31,34048.60,9530.56,...,1212.20,262.27,6002.29,2768.34,3233.95,3651.84,1377.64,1011.03,789.74,473.43
2005,9560767.52,11602700.52,310037.6,59149.71,59149.71,59149.71,177746.37,59542.36,36451.86,10003.21,...,1494.36,367.37,7204.12,3369.56,3834.56,4528.42,1707.52,1260.92,948.44,611.54
2006,10112652.86,12276152.86,325151.5,60680.96,60680.96,60680.96,187513.60,61811.21,37858.64,10513.57,...,1604.39,372.82,7797.05,3587.70,4209.35,4984.47,1891.07,1380.95,1041.52,670.93
2007,10738992.68,13003686.68,343618.9,63329.66,63329.66,63329.66,199409.06,65609.67,39919.68,11365.68,...,1791.08,425.78,8629.77,3926.75,4703.02,5533.55,2095.73,1500.61,1196.29,740.92
2008,11085402.06,13082100.06,351743.1,64250.68,64250.68,64250.68,203206.49,67124.35,41129.60,11431.88,...,1930.15,413.16,8979.14,4169.02,4810.13,5937.75,2204.05,1669.59,1258.33,805.77
2009,10587697.08,12324697.08,346472.8,65786.35,65786.35,65786.35,199520.43,65011.83,38499.78,11235.14,...,1677.22,409.18,8209.21,3762.87,4446.34,5508.64,2033.02,1453.03,1201.41,821.18


In [6]:
gdp['BE']

2000    256376.4
2001    264334.9
2002    273255.9
2003    281200.2
2004    296819.7
2005    310037.6
2006    325151.5
2007    343618.9
2008    351743.1
2009    346472.8
2010    363140.1
2011    375967.8
2012    386174.7
2013      392880
2014    403003.3
2015    416701.4
2016    430372.1
2017    446364.9
2018    459819.8
Name: BE, dtype: object